In [75]:
import pandas as pd
pd.set_option("display.width",1000)

xls = pd.ExcelFile('customer_factory_14499.xlsx')

# 如果你想读取所有的工作表，可以遍历sheet_names
all_dfs = {sheet_name: pd.read_excel('customer_factory_14499.xlsx', sheet_name=sheet_name) for sheet_name in pd.ExcelFile('customer_factory_14499.xlsx').sheet_names}

In [76]:
visit_factory = all_dfs["Event - Visit Factory"]
factory_product_click = all_dfs["Event - Factory Product Click"]
product_view = all_dfs["Event - Product View"]

In [77]:
# def factory_linkage(visit_factory,factory_product_click,product_view):
#     # Correcting the format for datetime conversion
#     visit_factory['event_timestamp'] = pd.to_datetime(visit_factory['event_timestamp'], format='%Y/%m/%d/%H/%M/%S')
#     factory_product_click['event_timestamp'] = pd.to_datetime(factory_product_click['event_timestamp'], format='%Y/%m/%d/%H/%M/%S')

#     # Perform the operation again with corrected datetime format
#     cross_joined_df = factory_product_click.assign(key=1).merge(
#         visit_factory.assign(key=1),
#         on='key',
#         suffixes=('_fp_click', '_visit')
#     ).query(
#         "customer_id_fp_click == customer_id_visit & "
#         "ga_session_id_fp_click == ga_session_id_visit & "
#         "event_timestamp_fp_click > event_timestamp_visit"
#     )

#     # Find the closest visit event for each product click event
#     closest_matches = cross_joined_df.groupby(['cookie_id_fp_click', 'event_timestamp_fp_click'], as_index=False).apply(
#         lambda x: x.loc[(x['event_timestamp_fp_click'] - x['event_timestamp_visit']).idxmin()]
#     ).reset_index(drop=True)

#     # Selecting relevant columns for clarity and removing duplicate columns from join
#     final_df = closest_matches[['report_date_fp_click', 'cookie_id_fp_click', 'customer_id_fp_click',
#                                 'ga_session_id_fp_click', 'event_timestamp_fp_click', 'event_name_fp_click',
#                                 'event_value_fp_click', 'event_timestamp_visit', 'event_value_visit']]

#     final_df = final_df.rename(columns={
#         'report_date_fp_click': 'report_date',
#         'cookie_id_fp_click': 'cookie_id',
#         'customer_id_fp_click': 'customer_id',
#         'ga_session_id_fp_click': 'ga_session_id',
#         'event_timestamp_fp_click': 'event_timestamp_click',
#         'event_name_fp_click': 'event_name',
#         'event_value_fp_click': 'factory_product_click',
#         'event_timestamp_visit': 'event_timestamp_visit',
#         'event_value_visit': 'visit_factory'
#     })

#     # Drop the 'event_name' column
#     final_df.drop('event_name', axis=1, inplace=True)

#     # Adjust the order of columns
#     final_df = final_df[[
#         'report_date', 
#         'customer_id', 
#         'cookie_id', 
#         'ga_session_id',	
#         'event_timestamp_visit',
#         'visit_factory',
#         'event_timestamp_click',
#         'factory_product_click'
#     ]]

#     result_df = visit_factory.merge(
#         final_df,
#         left_on=['customer_id', 'ga_session_id', 'event_timestamp'],
#         right_on=['customer_id', 'ga_session_id', 'event_timestamp_visit'],
#         how='left'
#     )

#     # Drop specified columns from the result DataFrame
#     result_df_dropped = result_df.drop(['report_date_y', 'cookie_id_y', 'event_timestamp_visit', 'visit_factory'], axis=1)

#     # Rename the 'event_value' column to 'visit_factory'
#     result_df_dropped = result_df_dropped.rename(columns={'event_value': 'visit_factory'})

#     # 分割event_value列
#     product_view[['product_name', 'product_id']] = product_view['event_value'].str.split('\|\|\|', expand=True)

#     # 删除原始的event_value列
#     product_view.drop(columns=['event_value'], inplace=True)

#     distinct_products = product_view[['product_name', 'product_id']].drop_duplicates()
#     distinct_products['product_name'] = distinct_products['product_name'].str.strip()

#     # 左连接result_df_dropped和distinct_products
#     merged_df = pd.merge(result_df_dropped, distinct_products, how='left', left_on='factory_product_click', right_on='product_name')

#     # 删除重复的product_name列
#     merged_df.drop(columns=['product_name'], inplace=True)

#     return merged_df

In [78]:
def factory_linkage(visit_factory,factory_product_click,product_view):
    # Correcting the format for datetime conversion
    visit_factory['event_timestamp'] = pd.to_datetime(visit_factory['event_timestamp'], format='%Y/%m/%d/%H/%M/%S')
    factory_product_click['event_timestamp'] = pd.to_datetime(factory_product_click['event_timestamp'], format='%Y/%m/%d/%H/%M/%S')

    # Perform the operation again with corrected datetime format and directly select relevant columns post-query
    cross_joined_df = factory_product_click.assign(key=1).merge(
        visit_factory.assign(key=1),
        on='key',
        suffixes=('_fp_click', '_visit')
    ).query(
        "customer_id_fp_click == customer_id_visit & "
        "ga_session_id_fp_click == ga_session_id_visit & "
        "event_timestamp_fp_click > event_timestamp_visit"
    )

    # Find the closest visit event for each product click event and directly select relevant columns
    final_df = cross_joined_df.groupby(['cookie_id_fp_click', 'event_timestamp_fp_click'], as_index=False).apply(
        lambda x: x.loc[(x['event_timestamp_fp_click'] - x['event_timestamp_visit']).idxmin()]
    ).reset_index(drop=True)[['report_date_fp_click', 'cookie_id_fp_click', 'customer_id_fp_click',
                              'ga_session_id_fp_click', 'event_timestamp_fp_click', 'event_name_fp_click',
                              'event_value_fp_click', 'event_timestamp_visit', 'event_value_visit']]

# 重命名、删除和重新排序列
    final_df = final_df.rename(columns={
        'report_date_fp_click': 'report_date',
        'cookie_id_fp_click': 'cookie_id',
        'customer_id_fp_click': 'customer_id',
        'ga_session_id_fp_click': 'ga_session_id',
        'event_timestamp_fp_click': 'event_timestamp_click',
        'event_value_fp_click': 'factory_product_click',
        'event_timestamp_visit': 'event_timestamp_visit',
        'event_value_visit': 'visit_factory'
    })[[
        'report_date', 
        'customer_id', 
        'cookie_id', 
        'ga_session_id',  
        'event_timestamp_visit',
        'visit_factory',
        'event_timestamp_click',
        'factory_product_click'
    ]]

# ------------------------------
    result_df = visit_factory.merge(
        final_df,
        left_on=['customer_id', 'ga_session_id', 'event_timestamp'],
        right_on=['customer_id', 'ga_session_id', 'event_timestamp_visit'],
        how='left'
    ).drop(['report_date_y', 'cookie_id_y', 'event_timestamp_visit', 'visit_factory'], axis=1) \
    .rename(columns={'event_value': 'visit_factory'})

    # 处理product_view中的event_value列，分割成product_name和product_id，然后删除原始的event_value列
    product_view[['product_name', 'product_id']] = product_view['event_value'].str.split('\|\|\|', expand=True)
    product_view = product_view.drop(columns=['event_value']).drop_duplicates(subset=['product_name', 'product_id'])
    product_view['product_name'] = product_view['product_name'].str.strip()

    # 左连接result_df和distinct_products，然后删除重复的product_name列
    merged_df = result_df.merge(
        product_view,
        how='left',
        left_on='factory_product_click',
        right_on='product_name'
    ).drop(columns=['product_name','event_name_x', 'report_date', 'cookie_id', 'customer_id_y', 'ga_session_id_y','event_timestamp_y','event_name_y'])

    merged_df.rename(columns={col: col.replace('_x', '') for col in merged_df.columns if '_x' in col}, inplace=True)

    return merged_df

In [79]:
A = factory_linkage(visit_factory, factory_product_click, product_view)
A.to_csv("BBB.csv")